In [ ]:
from rayleigh_diagnostics import AZ_Avgs, build_file_list, TimeAvg_AZAverages, plot_azav, streamfunction
import matplotlib.pyplot as plt
import numpy
import pylab

In [ ]:
pref = ['./ra1e6/']
cases=['ek1em3_ra1e6_fr0', 'ek1em3_ra1e6_fr0.1', 'ek1em3_ra1e6_fr0.5', 
       'ek1em3_ra1e6_fr1.0', 'ek1em3_ra1e6_fr1.5' ]
ncases = len(cases)
imin = 0
imax = 10000000 # 10 million
numavg = 3   # Number of AZ_Avg files

In [ ]:
comp_files=[]
for p in pref:
    ctemp=[]
    for c in cases:
        case_path = p+c
        files = build_file_list(imin,imax,path = case_path+'/AZ_Avgs')  # Build file list
        nf = len(files)   # grab first and last iter for compiled file  name
        files = files[nf-numavg:nf]
        nf = len(files)
        f0 = files[0].split('/')
        f1 = files[nf-1].split('/')
        i0 = f0[len(f0)-1]
        i1 = f1[len(f1)-1]
        comp_file=case_path+'/azcomp_'+i0+'_'+i1
        ctemp.append(comp_file)
        TimeAvg_AZAverages(files,comp_file)
    comp_files.append(ctemp)

In [ ]:
#Loop over the compiled files
azavgs = []
for cf in comp_files[0]:
    a = AZ_Avgs(cf,path='')
    azavgs.append(a)

In [ ]:
tsize = 10     # title font size
cbfsize = 10   # colorbar font size
plt.rcParams.update({'font.size': 14})

In [ ]:
#Temperature
%matplotlib inline
fig, ax = plt.subplots(ncols=ncases,figsize=(15,5))
units = '(nondimensional)'
for i,a in enumerate(azavgs):
    temp = a.vals[:,:,a.lut[501],0]
    radius   = a.radius
    costheta = a.costheta
    sintheta = a.sintheta
    nr=a.nr
    ntheta=a.ntheta
    #Subtrace the ell=0 component from temperature at each radius
    for j in range(nr):
        temp[:,j]=temp[:,j] - numpy.mean(temp[:,j])    
    print(temp.shape)
    plot_azav(fig,ax[i],temp,radius,costheta,sintheta,mycmap='RdYlBu_r',boundsfactor = 2, 
              boundstype='rms', units=units, fontsize = cbfsize)
    ax[i].set_title(cases[i],fontsize=tsize)

In [ ]:
#Omega
%matplotlib inline
fig, ax = plt.subplots(ncols=ncases,figsize=(15,5))
units = '(nondimensional)'
for i,a in enumerate(azavgs):
    vphi = a.vals[:,:,a.lut[3],0]
    radius   = a.radius
    costheta = a.costheta
    sintheta = a.sintheta
    nr=a.nr
    ntheta=a.ntheta
    #convert vphi to omega

    omega=numpy.zeros((ntheta,nr))
    for j in range(nr):
        omega[:,j]=vphi[:,j]/(radius[j]*sintheta[:])

    plot_azav(fig,ax[i],omega,radius,costheta,sintheta,mycmap='RdYlBu_r',boundsfactor = 2, 
              boundstype='rms', units=units, fontsize = cbfsize)
    ax[i].set_title(cases[i],fontsize=tsize)

In [ ]:
#Omega
%matplotlib inline
fig, ax = plt.subplots(ncols=ncases,figsize=(15,5))
units = '(nondimensional)'
for i,a in enumerate(azavgs):
    vr = a.vals[:,:,a.lut[1],0]
    vtheta = a.vals[:,:,a.lut[2],0]
    radius   = a.radius
    costheta = a.costheta
    sintheta = a.sintheta
    nr=a.nr
    ntheta=a.ntheta
    #build streamfunction + magnitude array
    psi = streamfunction(vr,vtheta,radius,costheta,order=0)
    vmag = numpy.sqrt(vr**2+vtheta**2)*numpy.sign(psi)
    plot_azav(fig,ax[i],psi,radius,costheta,sintheta,mycmap='RdYlBu_r',boundsfactor = 2, 
              boundstype='rms', units=units, fontsize = cbfsize, underlay=vmag)
    ax[i].set_title(cases[i],fontsize=tsize)